# Python Generative AI Youtube Video Creator

**Created by Ayden Salazar, AI Engineer**

Run the cells in this notebook in series to generate a scary storytelling Youtube video. Follow along the step-by-step guide to customize the video to your desire.

### Import necessary utility functions

In [44]:
from storytelling_utils import *

### Fetch API keys from file

This notebook will not run unless you enter your OpenAI and ElevenLabs API keys into the `api_keys.json` file.

The format should look like the following:

`{ 
    "api_keys": {
      "openai": "insert_api_key_here",
      "elevenlabs": "insert_api_key_here"
    }
}`

In [45]:
api_key_dict = retrieve_api_key_dict("api_keys.json")

In [46]:
# OpenAI API Key
openai.api_key = api_key_dict['openai']

# Step 1. Generate Prompt

The function `generate_story_prompt()` helps generate story prompts for scary stories. You can think of it as a tool to provide a starting point for writing a spooky tale. Here's how it works:

- When you use this function, you can provide a "prompt" as input, which is basically a specific topic or idea for the scary story. If you give it a prompt, the function takes that prompt and creates a story prompt based on it. This prompt includes a request to write the beginning of a realistic scary story (around 4000 characters long) that's told from a first-person perspective. The topic you provided will be a part of this story prompt.

- On the other hand, if you don't provide a prompt, the function reads from a file called "story_topics.csv." This file contains different story topics, some of which might have been used before and some that haven't been used yet. The function looks for the next unused story topic in this file and extracts it.

- Then, similar to the prompt case, the function builds a story prompt using this extracted topic. It's the same request: write the beginning of a 4000-character scary story, told in the first person, based on this topic. Additionally, the function marks the extracted topic as "used" by changing a "Published" column in the file to "True." This helps keep track of which topics have been used already.

- Finally, the updated list of story topics is saved back to the "story_topics.csv" file, so that next time, the function knows which topics have been used and which ones are still available.

In [47]:
# Generate story prompt
user_content, extracted_topic = generate_story_prompt(prompt='music concert')

# Step 2. Generate Stories

The function `generate_short_story` takes a text prompt provided by the user and uses it to create a short story. Here's a breakdown of how it works:

1. **Function Signature and Description**: The function is defined with the name `generate_short_story`. It takes a single input parameter `user_content`, which should be a string representing the user's prompt for the story.

2. **Initializing Story Text**: The function starts by initializing an empty string called `story_text`. This string will be used to store the generated story.

3. **Generating the Beginning**: The function then constructs a list called `messages` containing a single message. This message represents the user's prompt. The `openai.ChatCompletion.create` method is used to generate a response from the model based on this message. The model used for generation is `gpt-3.5-turbo`. The response (reply) from the model is extracted and added to the `story_text`.

4. **Generating the Ending**: The generated reply is added to the `messages` list as an assistant's message. A new user message is added to instruct the assistant to provide the ending of the story. Another call to `openai.ChatCompletion.create` is made using the updated `messages` list. The response generated by the model is again extracted and added to the `story_text`.

5. **Returning the Story**: After generating both the beginning and ending of the story, the function returns the `story_text`, which now contains the complete short story.

6. **Printing for Visualization**: Throughout the process, the function prints out the model-generated replies with labels like "ChatGPT:" to show the conversation between the user and the model. This helps visualize how the story is being generated step by step.

In [48]:
# Generate 3 stories
story_dict = generate_n_stories_dict(user_content, n=3)

Generating story...
ChatGPT: I can still remember the feeling of excitement and anticipation that pulsated through my veins as I made my way through the crowded city streets towards the music venue. The sun had just set, casting a pale blue hue over everything as people flooded the sidewalks, their outfits a mixture of trendy bands shirts and risqué clothing.

I had been looking forward to this concert for weeks, eagerly counting down the days until I could dance and scream my heart out to some of my favorite music. It wasn't until I got closer to the venue that I started to feel a tinge of unease creeping up my spine.

The venue itself was an old, decrepit building that looked like it had been abandoned for years before being renovated into a concert hall. A banner with the name of the band hung over the front entrance, the neon lights causing it to flicker eerily in the night sky.

I had been to plenty of sketchy venues before, but as I approached the entrance, there was something in

# Step 3. Clean, chunk, and save the stories

Here's an explanation of the function `clean_chunk_save`:

1. **Function Signature and Description**: The function is named `clean_chunk_save`. It takes three parameters: `story_dict`, a dictionary containing stories; `extracted_topic`, a string representing the topic of the stories; and an optional parameter `n` with a default value of 4999, which represents the desired chunk length. The purpose of the function is to clean the text in the stories, break them into smaller chunks, save the chunks as .txt files, and return the updated story dictionary.

2. **Defining Filepaths**: The function begins by defining two filepaths, `filepath` and `text_filepath`, for organizing the saved data files. These paths are based on the provided `extracted_topic`.

3. **Creating Directories**: It then creates two directories using the defined filepaths: the main directory and the text directory. The `os.mkdir` function is used for this purpose.

4. **Iterating Over Stories**: The function iterates through each story in the provided `story_dict`.

5. **Cleaning the Story**: For each story, it calls a function `remove_newlines_backslashes` to clean the text by removing newlines and backslashes. The cleaned text is stored in `story_text_cleaned`.

6. **Chunking the Text**: The cleaned text is then passed to a function called `chunk_text`, along with the desired chunk length `n`. The purpose of this is to break the story into smaller chunks of approximately the specified length.

7. **Updating the Dictionary**: The resulting chunks are reassigned to the corresponding story in the `story_dict`.

8. **Saving Chunks to Files**: The function saves the first chunk of each story as a .txt file in the text directory. It iterates through the stories, creates a file named based on the index, and writes the content of the first chunk to the file.

9. **Saving Dictionary as JSON**: After processing all stories, the function saves the updated `story_dict` as a JSON file named 'cleaned_story_dict.json' using the `json.dumps` function.

10. **Returning the Updated Dictionary**: Finally, the function returns the `story_dict` after processing and saving.

In summary, the `clean_chunk_save` function takes a dictionary of stories, cleans their text by removing specific characters, divides them into smaller chunks, saves the chunks as .txt files in directories based on the topic, and returns the updated dictionary. This function is useful for organizing and preprocessing a collection of stories for further analysis or use.

In [49]:
# Clean, chunk, and save the stories
cleaned_story_dict = clean_chunk_save(story_dict, extracted_topic, n=4999)

# Step 4. Generate Story Narration

The `generate_narration` function performs the following tasks:

1. Creates a user object for text-to-speech using an API key.
2. Initializes a voice object for the desired voice.
3. Defines a filepath for audio files based on the provided `extracted_topic`.
4. Creates an audio directory if it doesn't exist.
5. Iterates through stories in the `story_dict`.
6. For each story, iterates through its chunks.
7. Generates audio from each chunk using the defined voice.
8. Cleans the story title for use in filenames.
9. Exports the generated audio as an mp3 file using a formatted filename.

In summary, the function uses a user's API key to generate audio narration for a given collection of stories. The narration is generated in parts corresponding to the story chunks, and the audio files are saved in a specified directory based on the story titles.

In [50]:
# Generate story narration
generate_narration(cleaned_story_dict, api_key_dict['elevenlabs'], extracted_topic)

# Step 5. Generate Story Images

The `generate_images` function does the following:

1. Initializes an empty dictionary `image_object_dict`.
2. Iterates through each story in the provided `story_dict`.
3. For each story, generates a list of image objects based on picture prompts.
4. The function generates a given number (`n`) of image objects for each story.
5. Each image object is created using a prompt related to the extracted topic.
6. The generated image objects are added to the `image_object_list` for each story.
7. After generating all image objects for a story, the function waits for 15 seconds (using `time.sleep(15)`) before moving to the next story. This might be to prevent overloading the API.
8. The `image_object_list` is printed.
9. The list of image objects for each story is stored in the `image_object_dict`.
10. Finally, the function returns the `image_object_dict`, which contains the generated image objects associated with each story.

In summary, the `generate_images` function uses OpenAI's API to create a specified number of images for each story in the given dictionary. It generates these images based on prompts related to the extracted topic and stores the resulting image objects in a dictionary.

In [51]:
# Generate story images
generate_images_pipeline(cleaned_story_dict, extracted_topic, n=6)

[<OpenAIObject at 0x7f9e91b3c860> JSON: {
  "created": 1686456990,
  "data": [
    {
      "url": "https://oaidalleapiprodscus.blob.core.windows.net/private/org-p8NoTe2TJj38Z37I2lSgNOP4/user-uXpor5bDfLQGnelyZj2BfrlB/img-Avq7d1IugXFbV85pMNYjOEhV.png?st=2023-06-11T03%3A16%3A30Z&se=2023-06-11T05%3A16%3A30Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-06-10T06%3A09%3A58Z&ske=2023-06-11T06%3A09%3A58Z&sks=b&skv=2021-08-06&sig=7wf6Dn1Yhdw/CmiBTj05KLnuVt3CpyzQrlv9LZ812s4%3D"
    }
  ]
}, <OpenAIObject at 0x7f9e91b3cdb0> JSON: {
  "created": 1686457012,
  "data": [
    {
      "url": "https://oaidalleapiprodscus.blob.core.windows.net/private/org-p8NoTe2TJj38Z37I2lSgNOP4/user-uXpor5bDfLQGnelyZj2BfrlB/img-HbBBxrKVJvH2LgpCeBu349dY.png?st=2023-06-11T03%3A16%3A52Z&se=2023-06-11T05%3A16%3A52Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e

# Step 6. Generate Video

The `generate_video` function performs the following tasks:

1. Creates a directory to save the video files.
2. Defines the screen size for the video.
3. Loads and adjusts a scary music audio file.
4. Creates an empty list `clips` to store video clips.
5. Sets the initial time for tracking the video's current position.
6. Iterates through each story in the `story_dict`.
7. For each story:
   - Creates a text clip with the story title.
   - Sets the duration for the text clip.
   - Retrieves image files from a directory.
   - Appends the text clip to the `clips` list.
   - Creates an audio clip from a corresponding audio file.
   - Sets the audio clip's start time based on the previous audio's end time.
   - Determines image files related to the story section.
   - Calculates the duration for each image clip.
   - Creates a list of image clips and combines them into a video clip.
   - Combines the image clip with the associated audio clip.
   - Appends the combined clip to the `clips` list.
   - Updates time tracking variables.
8. Concatenates all the clips into a single video using `CompositeVideoClip`.
9. Sets the final duration of the video, including a black screen time.
10. Makes the scary music loop for the duration of the video.
11. Combines the original audio and the scary music to create composite audio.
12. Combines the video and composite audio.
13. Writes the final video to a file.

In summary, the `generate_video` function creates a video by combining text clips, image clips, and audio clips based on provided story data. It uses image prompts, story titles, and corresponding audio to construct a narrative video. The final video is then written to a specified file location.

In [52]:
# Generate video
generate_video(cleaned_story_dict, extracted_topic)

Moviepy - Building video data/music concert/video/output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video data/music concert/video/output.mp4



Moviepy - Done !
Moviepy - video ready data/music concert/video/output.mp4


: 